# Prepare environment + data

In [ ]:
import argparse
import pickle
from collections import Counter
from pathlib import Path

import face_recognition
from PIL import Image, ImageDraw

In [5]:
import os
from pathlib import Path

def rename_images_in_subfolders(directory):
    # Convert to a Path object for easier manipulation
    training_path = Path(directory)
    
    # Iterate through each subfolder in the training directory
    for subfolder in training_path.iterdir():
        if subfolder.is_dir():
            # Collect existing filenames in the subfolder
            existing_filenames = {int(f.stem) for f in subfolder.glob("*.jpg") if f.stem.isdigit()}
            
            # Start renaming process
            counter = 1
            for image_file in subfolder.glob("*.jpg"):
                print(image_file)
                if len(image_file.stem) > 10 or not image_file.stem.isdigit():
                    # Find the next available number to rename to
                    while counter in existing_filenames:
                        counter += 1

                    # Create new filename
                    new_filename = subfolder / f"{counter}.jpg"
                    
                    # Rename the file
                    image_file.rename(new_filename)

                    # Update existing filenames set
                    existing_filenames.add(counter)
                    counter += 1
# print(os.getcwd())
# rename_images_in_subfolders("training")

c:\Users\alexa\Documents\CODING PROJECTS\UCD RESEARCH\ucd_research_facialrecognition\1_StaticFacialRecognition\attempt2
training\ben_affleck\1.jpg
training\ben_affleck\10.jpg
training\ben_affleck\11.jpg
training\ben_affleck\12.jpg
training\ben_affleck\13.jpg
training\ben_affleck\14.jpg
training\ben_affleck\2.jpg
training\ben_affleck\3.jpg
training\ben_affleck\4.jpg
training\ben_affleck\5.jpg
training\ben_affleck\6.jpg
training\ben_affleck\7.jpg
training\ben_affleck\8.jpg
training\ben_affleck\9.jpg
training\elon_musk\161854.jpg
training\elon_musk\161860.jpg
training\elon_musk\161869.jpg
training\elon_musk\161871.jpg
training\elon_musk\161877.jpg
training\elon_musk\161879.jpg
training\elon_musk\161881.jpg
training\elon_musk\161888.jpg
training\elon_musk\161889.jpg
training\elton_john\1.jpg
training\elton_john\10.jpg
training\elton_john\11.jpg
training\elton_john\12.jpg
training\elton_john\13.jpg
training\elton_john\14.jpg
training\elton_john\15.jpg
training\elton_john\2.jpg
training\elto

### Setting up environment + directories

In [ ]:
DEFAULT_ENCODINGS_PATH = Path("output/encodings.pkl")
BOUNDING_BOX_COLOR = "blue"
TEXT_COLOR = "white"

# Create directories if they don't already exist
Path("training").mkdir(exist_ok=True)
Path("output").mkdir(exist_ok=True)
Path("validation").mkdir(exist_ok=True)

In [ ]:
parser = argparse.ArgumentParser(description="Recognize faces in an image")
parser.add_argument("--train", action="store_true", help="Train on input data")
parser.add_argument(
    "--validate", action="store_true", help="Validate trained model"
)
parser.add_argument(
    "--test", action="store_true", help="Test the model with an unknown image"
)
parser.add_argument(
    "-m",
    action="store",
    default="hog",
    choices=["hog", "cnn"],
    help="Which model to use for training: hog (CPU), cnn (GPU)",
)
parser.add_argument(
    "-f", action="store", help="Path to an image with an unknown face"
)
args = parser.parse_args()

### Helper Methods

In [ ]:
def _recognize_face(unknown_encoding, loaded_encodings):
    """
    Given an unknown encoding and all known encodings, find the known
    encoding with the most matches.
    """
    boolean_matches = face_recognition.compare_faces(
        loaded_encodings["encodings"], unknown_encoding
    )
    votes = Counter(
        name
        for match, name in zip(boolean_matches, loaded_encodings["names"])
        if match
    )
    if votes:
        return votes.most_common(1)[0][0]

In [ ]:
def _display_face(draw, bounding_box, name):
    """
    Draws bounding boxes around faces, a caption area, and text captions.
    """
    top, right, bottom, left = bounding_box
    draw.rectangle(((left, top), (right, bottom)), outline=BOUNDING_BOX_COLOR)
    text_left, text_top, text_right, text_bottom = draw.textbbox(
        (left, bottom), name
    )
    draw.rectangle(
        ((text_left, text_top), (text_right, text_bottom)),
        fill=BOUNDING_BOX_COLOR,
        outline=BOUNDING_BOX_COLOR,
    )
    draw.text(
        (text_left, text_top),
        name,
        fill=TEXT_COLOR,
    )

### Encode Data

In [ ]:
def encode_known_faces(
    model: str = "hog", encodings_location: Path = DEFAULT_ENCODINGS_PATH
) -> None:
    """
    Loads images in the training directory and builds a dictionary of their
    names and encodings.
    """
    names = []
    encodings = []

    for filepath in Path("training").glob("*/*"):
        name = filepath.parent.name
        image = face_recognition.load_image_file(filepath)

        face_locations = face_recognition.face_locations(image, model=model)
        face_encodings = face_recognition.face_encodings(image, face_locations)

        for encoding in face_encodings:
            names.append(name)
            encodings.append(encoding)

    name_encodings = {"names": names, "encodings": encodings}
    with encodings_location.open(mode="wb") as f:
        pickle.dump(name_encodings, f)

In [ ]:
def recognize_faces(
    image_location: str,
    model: str = "hog",
    encodings_location: Path = DEFAULT_ENCODINGS_PATH,
) -> None:
    """
    Given an unknown image, get the locations and encodings of any faces and
    compares them against the known encodings to find potential matches.
    """
    with encodings_location.open(mode="rb") as f:
        loaded_encodings = pickle.load(f)

    input_image = face_recognition.load_image_file(image_location)

    input_face_locations = face_recognition.face_locations(
        input_image, model=model
    )
    input_face_encodings = face_recognition.face_encodings(
        input_image, input_face_locations
    )

    pillow_image = Image.fromarray(input_image)
    draw = ImageDraw.Draw(pillow_image)

    for bounding_box, unknown_encoding in zip(
        input_face_locations, input_face_encodings
    ):
        name = _recognize_face(unknown_encoding, loaded_encodings)
        if not name:
            name = "Unknown"
        _display_face(draw, bounding_box, name)

    del draw
    pillow_image.show()

In [ ]:
def validate(model: str = "hog"):
    """
    Runs recognize_faces on a set of images with known faces to validate
    known encodings.
    """
    for filepath in Path("validation").rglob("*"):
        if filepath.is_file():
            recognize_faces(
                image_location=str(filepath.absolute()), model=model
            )

In [ ]:
if __name__ == "__main__":
    if args.train:
        encode_known_faces(model=args.m)
    if args.validate:
        validate(model=args.m)
    if args.test:
        recognize_faces(image_location=args.f, model=args.m)